<a href="https://colab.research.google.com/github/ba18406/Lab001/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv")

In [0]:
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
print (df)

        74  -7.82   8.79  -9.66  -8.16  ...  99.22  -5.63  99.23  99.24  99.25
0      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
1       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
2       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
4      100  -6.17  -3.54   0.44  -8.50  ...  -3.35   0.05  -9.08  -5.05  -3.45
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24978   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24979   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24980   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24982 rows x 101 columns]


In [0]:
arr = df.values
print(arr.shape)

(24982, 101)


In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1835282 (1835282,)


In [0]:
def replace(orig, percentage=0.1):
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr[idx[0][0], idx[1][0]]

0.68

In [0]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
#Latent Factor
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
n_latent_factors = 2

user_ratings = df
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
latent_item_features

array([[8.126e-01, 5.855e-01],
       [4.546e-01, 2.174e-01],
       [1.847e-01, 3.824e-01],
       [3.322e-01, 3.349e-01],
       [5.965e-01, 2.775e-01],
       [4.989e-01, 8.284e-01],
       [3.505e-01, 7.598e-01],
       [2.307e-01, 7.584e-01],
       [6.778e-01, 3.935e-01],
       [3.177e-01, 3.504e-02],
       [8.860e-01, 6.871e-01],
       [5.827e-01, 7.257e-01],
       [2.094e-01, 1.020e-01],
       [9.959e-01, 1.563e-01],
       [4.210e-01, 1.017e-01],
       [8.049e-01, 6.136e-03],
       [3.139e-01, 9.740e-01],
       [8.640e-01, 9.899e-01],
       [1.265e-01, 7.736e-01],
       [2.982e-01, 2.483e-01],
       [6.065e-01, 8.493e-01],
       [9.003e-01, 3.447e-01],
       [3.404e-01, 6.956e-01],
       [9.545e-01, 3.320e-01],
       [4.338e-01, 6.201e-01],
       [6.590e-01, 9.375e-01],
       [5.674e-01, 2.142e-01],
       [9.344e-01, 8.447e-01],
       [4.393e-01, 1.942e-01],
       [7.634e-01, 2.323e-01],
       [4.474e-01, 5.630e-01],
       [3.063e-01, 9.823e-01],
       [

In [0]:
latent_user_preferences

array([[0.688, 0.861],
       [0.843, 0.175],
       [0.399, 0.287],
       ...,
       [0.304, 0.159],
       [0.04 , 0.548],
       [0.933, 0.473]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[1.064, 0.5  , 0.456, ..., 0.256, 0.826, 0.752],
       [0.788, 0.421, 0.223, ..., 0.094, 0.853, 0.784],
       [0.492, 0.244, 0.183, ..., 0.095, 0.44 , 0.402],
       ...,
       [0.34 , 0.173, 0.117, ..., 0.058, 0.325, 0.298],
       [0.353, 0.137, 0.217, ..., 0.139, 0.138, 0.121],
       [1.035, 0.527, 0.353, ..., 0.174, 0.995, 0.911]])